In [69]:
import numpy  as np
import pandas as pd 
import scipy  as sp

from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn.linear_model as lm
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split

In [70]:
data = pd.read_csv("..\\data\\test_questions.csv")
data.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [71]:
data['question1'] = data['question1'].str.lower()
data['question2'] = data['question2'].str.lower()
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0


In [72]:
data['question1'] = data['question1'].str.replace(r'[^\w\s]+', '')
data['question2'] = data['question2'].str.replace(r'[^\w\s]+', '')
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


In [73]:
X = data.drop(['is_duplicate'], axis=1)
y = data['is_duplicate']
train, test, y_train, y_test = train_test_split(X, y, test_size=0.45, random_state=241)

In [74]:
tfv = TfidfVectorizer(min_df=2,  max_features=None, strip_accents='unicode',  
      analyzer='word',ngram_range=(1, 2), 
      use_idf=1,smooth_idf=1,sublinear_tf=1)

In [75]:
tfv.fit(pd.concat([train['question1'],train['question2']]))

TfidfVectorizer(min_df=2, ngram_range=(1, 2), smooth_idf=1,
                strip_accents='unicode', sublinear_tf=1, use_idf=1)

In [76]:
tr1 = tfv.transform(train['question1']) 
tr2 = tfv.transform(train['question2'])
    
ts1 = tfv.transform(test['question1']) 
ts2 = tfv.transform(test['question2'])

X = sp.sparse.hstack([tr1,tr2])
Z = sp.sparse.hstack([ts1,ts2])

In [77]:
X = sp.sparse.hstack([tr1,tr2])
Z = sp.sparse.hstack([ts1,ts2])

print (X.shape)
print (Z.shape)
print (y_train.shape)
print (y_test.shape)
print (y.shape)

(54997, 236222)
(44999, 236222)
(54997,)
(44999,)
(99996,)


In [78]:
clf = lm.LogisticRegression(C=1, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1.0, max_iter=1000, multi_class='ovr', n_jobs=1,
        penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
        verbose=1, warm_start=False)

In [79]:
def rmse_cv(model):
    rmse= ms.cross_val_score(model, X, y_train, scoring="neg_log_loss", cv = 10)
    return(rmse)

In [80]:
rmse_cv(clf).mean()

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

-0.5107285026327151

In [81]:
clf.fit(X,y_train)

[LibLinear]

LogisticRegression(C=1, dual=True, intercept_scaling=1.0, max_iter=1000,
                   multi_class='ovr', n_jobs=1, solver='liblinear', verbose=1)

In [82]:
#Prediction

p_test = clf.predict_proba(Z)[:,1]
p_test

array([0.31903923, 0.87287407, 0.69589319, ..., 0.2389944 , 0.31093946,
       0.77850473])

In [84]:
clf.score(Z, y_test)

0.758594635436343

In [85]:
y_test

62671    1
85204    1
49928    1
48563    0
6757     1
        ..
21238    1
12571    0
49095    0
19780    1
45930    1
Name: is_duplicate, Length: 44999, dtype: int64

In [86]:
Z

<44999x236222 sparse matrix of type '<class 'numpy.float64'>'
	with 1461522 stored elements in COOrdinate format>